In [1]:
import numpy as np
import pandas as pd

# Collect data

### demographics

In [2]:
#user= dempgraphics?
users=pd.read_csv('demographics.csv')
users.columns=['Customer ID','Count','Gender','Age','Under_30','Senior_Citizen','Married','Dependents','Number_of_Dependents']
# do we still need Under 30/ Senior Citizen, or we need to group the age ?
users.drop(['Count'],axis=1,inplace=True)
users

,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents
0,1746-TGTWV,Male,78.0,No,Yes,No,No,0.0
1,8499-BRXTD,Female,74.0,No,Yes,Yes,NaN,1.0
2,2041-JIJCI,Male,71.0,No,Yes,No,Yes,3.0
3,5832-EXGTT,Female,78.0,No,Yes,Yes,Yes,1.0
4,8350-NYMVI,NaN,80.0,No,Yes,NaN,Yes,1.0
...,...,...,...,...,...,...,...,...
6158,7609-NRNCA,Female,NaN,No,No,No,No,0.0
6159,4835-YSJMR,Male,38.0,No,No,NaN,Yes,2.0
6160,5067-DGXLL,NaN,30.0,No,No,Yes,Yes,2.0
6161,3129-AAQOU,Female,32.0,No,No,Yes,Yes,2.0


### location

In [3]:
location=pd.read_csv('location.csv')
location.columns=['Customer ID','Count','Country','State','City','Zip Code','Lat Long','Latitude','Longtitude']
# Country, State are the same, latitude/longtitude's information is in Zip code
location.drop(['Count','Country','State','Lat Long','Latitude','Longtitude'],axis=1,inplace=True)

### population

In [4]:
population=pd.read_csv('population.csv')
population.columns=['ID','Zip Code','Population']
population.drop(['ID'],axis=1,inplace=True)
population

,Zip Code,Population
0,90001,54492
1,90002,44586
2,90003,58198
3,90004,67852
4,90005,43019
...,...,...
1666,96145,4002
1667,96146,942
1668,96148,678
1669,96150,33038


### merge location and population ,and join users

In [5]:
location=pd.merge(location,population,on='Zip Code')
users=pd.merge(users,location,on='Customer ID',how='outer')
users

,Customer ID,Gender,Age,Under_30,Senior_Citizen,Married,Dependents,Number_of_Dependents,City,Zip Code,Population
0,1746-TGTWV,Male,78.0,No,Yes,No,No,0.0,Los Angeles,90022.0,68701.0
1,8499-BRXTD,Female,74.0,No,Yes,Yes,NaN,1.0,NaN,NaN,NaN
2,2041-JIJCI,Male,71.0,No,Yes,No,Yes,3.0,NaN,NaN,NaN
3,5832-EXGTT,Female,78.0,No,Yes,Yes,Yes,1.0,Inglewood,90303.0,27778.0
4,8350-NYMVI,NaN,80.0,No,Yes,NaN,Yes,1.0,Whittier,90602.0,26265.0
...,...,...,...,...,...,...,...,...,...,...,...
6853,6839-ITVZJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jacumba,91934.0,699.0
6854,6821-JPCDC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Glennville,93226.0,296.0
6855,5799-JRCZO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bakersfield,93311.0,20440.0
6856,8741-LQOBK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mccloud,96057.0,1586.0


satisfaction

In [6]:
satisfaction=pd.read_csv('satisfaction.csv')
satisfaction.columns=['Customer ID','score']
users=pd.merge(users,satisfaction,on='Customer ID',how='outer')

services

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
services=pd.read_csv('services.csv')
services.columns=['Customer ID','Count','Quarter','Referred_a_Friend','Number of Referrals','Tenure in Months','Offer','Phone_Service','Avg Monthly Long Distance Charges','Multiple_Lines','Internet_Service','Internet_Type','Avg Monthly GB Download','Online_Security','Online_Backup','Device_Protection_Plan','Premium_Tech_Support','Streaming_TV','Streaming_Movies','Streaming_Music','Unlimited_Data','Contract','Paperless_Billing','Payment_Method','Monthly Charge','Total Charges','Total Refunds','Total Extra Data Charges','Total Long Distance Charges','Total Revenue']
# Quarter are always Q3
services.drop(['Count','Quarter'],axis=1,inplace=True)
users=pd.merge(users,services,on='Customer ID',how='outer')
#columns_to_encode = [1,3,4,5,6,8,12,13,14,15,16,17,18,20,21,22,23,34,35,36,27,28,29]
#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), columns_to_encode)], remainder='passthrough')
users = pd.concat((users,pd.get_dummies(users.Married,prefix='Married')),1)
users = pd.concat((users,pd.get_dummies(users.Gender,prefix='Gender')),1)
users = pd.concat((users,pd.get_dummies(users.Under_30,prefix='Under_30')),1)
users = pd.concat((users,pd.get_dummies(users.Senior_Citizen,prefix='Senior_Citizen')),1)
users = pd.concat((users,pd.get_dummies(users.Dependents,prefix='Dependents')),1)
users = pd.concat((users,pd.get_dummies(users.City,prefix='City')),1)
#users = pd.concat((users,pd.get_dummies(users.Referred_a_friend,prefix='Referred_a_friend')),1)
users = pd.concat((users,pd.get_dummies(users.Offer,prefix='Offer')),1)
users = pd.concat((users,pd.get_dummies(users.Phone_Service,prefix='Phone_Service')),1)
users = pd.concat((users,pd.get_dummies(users.Multiple_Lines,prefix='Multiple_Lines')),1)
users = pd.concat((users,pd.get_dummies(users.Internet_Service,prefix='Internet_Service')),1)
users = pd.concat((users,pd.get_dummies(users.Internet_Type,prefix='Internet_Type')),1)
users = pd.concat((users,pd.get_dummies(users.Online_Security,prefix='Online_Security')),1)
users = pd.concat((users,pd.get_dummies(users.Online_Backup,prefix='Online_Backup')),1)
users = pd.concat((users,pd.get_dummies(users.Device_Protection_Plan,prefix='Device_Protection_Plan')),1)
users = pd.concat((users,pd.get_dummies(users.Premium_Tech_Support,prefix='Premium_Tech_Support')),1)
users = pd.concat((users,pd.get_dummies(users.Streaming_TV,prefix='Streaming_TV')),1)
users = pd.concat((users,pd.get_dummies(users.Streaming_Movies,prefix='Streaming_Movies')),1)
users = pd.concat((users,pd.get_dummies(users.Streaming_Music,prefix='Streaming_Music')),1)
users = pd.concat((users,pd.get_dummies(users.Unlimited_Data,prefix='Unlimited_Data')),1)
users = pd.concat((users,pd.get_dummies(users.Contract,prefix='Contract')),1)
users = pd.concat((users,pd.get_dummies(users.Paperless_Billing,prefix='Paperless_Billing')),1)
users = pd.concat((users,pd.get_dummies(users.Payment_Method,prefix='Payment_Method')),1)

users.drop(['Married','Gender','Under_30','Senior_Citizen','Dependents','City','Phone_Service','Multiple_Lines','Internet_Service','Internet_Type','Online_Security','Online_Backup','Device_Protection_Plan','Premium_Tech_Support','Streaming_TV','Streaming_Movies','Streaming_Music','Unlimited_Data','Contract','Paperless_Billing','Payment_Method'],axis=1,inplace=True)
'''
df["Married"]=pd.util.hash_array(df["Married"].to_numpy())
df["Gender"]=pd.util.hash_array(df["Gender"].to_numpy())
df["Under 30"]=pd.util.hash_array(df["Under 30"].to_numpy())
df["Senior Citizen"]=pd.util.hash_array(df["Senior Citizen"].to_numpy())
df["Dependents"]=pd.util.hash_array(df["Dependents"].to_numpy())
df["City"]=pd.util.hash_array(df["City"].to_numpy())
#df["Referred a friend"]=pd.util.hash_array(df["Referred a friend"].to_numpy())
df["Offer"]=pd.util.hash_array(df["Offer"].to_numpy())
df["Phone Service"]=pd.util.hash_array(df["Phone Service"].to_numpy())
df["Multiple Lines"]=pd.util.hash_array(df["Multiple Lines"].to_numpy())
df["Internet Service"]=pd.util.hash_array(df["Internet Service"].to_numpy())
df["Internet Type"]=pd.util.hash_array(df["Internet Type"].to_numpy())
df["Online Security"]=pd.util.hash_array(df["Online Security"].to_numpy())
df["Online Backup"]=pd.util.hash_array(df["Online Backup"].to_numpy())
df["Device Protection Plan"]=pd.util.hash_array(df["Device Protection Plan"].to_numpy())
df["Premium Tech Support"]=pd.util.hash_array(df["Premium Tech Support"].to_numpy())
df["Streaming TV"]=pd.util.hash_array(df["Streaming TV"].to_numpy())
df["Streaming Movies"]=pd.util.hash_array(df["Streaming Movies"].to_numpy())
df["Streaming Music"]=pd.util.hash_array(df["Streaming Music"].to_numpy())
df["Unlimited Data"]=pd.util.hash_array(df["Unlimited Data"].to_numpy())
df["Contract"]=pd.util.hash_array(df["Contract"].to_numpy())
df["Paperless Billing"]=pd.util.hash_array(df["Paperless Billing"].to_numpy())
df["Payment Method"]=pd.util.hash_array(df["Payment Method"].to_numpy())
'''
users

/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_4127/1711089820.py:10: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  users = pd.concat((users,pd.get_dummies(users.Married,prefix='Married')),1)
/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_4127/1711089820.py:11: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  users = pd.concat((users,pd.get_dummies(users.Gender,prefix='Gender')),1)
/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_4127/1711089820.py:12: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  users = pd.concat((users,pd.get_dummies(users.Under_30,prefix='Under_30')),1)
/var/folders/sm/dnfqlxvd0d92nydq65rfrf980000gn/T/ipykernel_4127/1711089820.py:13: FutureWarning: In a future version of pandas all arguments of concat except

,Customer ID,Age,Number_of_Dependents,Zip Code,Population,score,Referred_a_Friend,Number of Referrals,Tenure in Months,Offer,...,Unlimited_Data_No,Unlimited_Data_Yes,Contract_Month-to-Month,Contract_One Year,Contract_Two Year,Paperless_Billing_No,Paperless_Billing_Yes,Payment_Method_Bank Withdrawal,Payment_Method_Credit Card,Payment_Method_Mailed Check
0,1746-TGTWV,78.0,0.0,90022.0,68701.0,NaN,No,0.0,1.0,None,...,1,0,1,0,0,0,1,1,0,0
1,8499-BRXTD,74.0,1.0,NaN,NaN,3.0,Yes,1.0,8.0,Offer E,...,0,1,1,0,0,0,1,0,0,0
2,2041-JIJCI,71.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,5832-EXGTT,78.0,1.0,90303.0,27778.0,2.0,Yes,1.0,25.0,Offer C,...,0,1,1,0,0,0,1,1,0,0
4,8350-NYMVI,80.0,1.0,90602.0,26265.0,2.0,Yes,1.0,37.0,Offer C,...,0,1,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7037,0516-WJVXC,NaN,NaN,NaN,NaN,NaN,Yes,2.0,1.0,None,...,0,0,1,0,0,1,0,0,1,0
7038,0388-EOPEX,NaN,NaN,NaN,NaN,NaN,No,0.0,35.0,NaN,...,1,0,0,1,0,1,0,0,1,0
7039,7055-JCGNI,NaN,NaN,NaN,NaN,NaN,No,0.0,NaN,Offer B,...,0,1,0,0,1,1,0,0,1,0
7040,6891-JPYFF,NaN,NaN,NaN,NaN,NaN,No,0.0,1.0,Offer E,...,0,1,1,0,0,0,1,0,1,0


status

In [8]:
status=pd.read_csv('status.csv')
status.columns=['Customer ID','Churn Category']
status['ans'] = status['Churn Category']
status.loc[status.ans=='No Churn','ans']='0'
status.loc[status.ans=='Competitor','ans']='1'
status.loc[status.ans=='Dissatisfaction','ans']='2'
status.loc[status.ans=='Attitude','ans']='3'
status.loc[status.ans=='Price','ans']='4'
status.loc[status.ans=='Other','ans']='5'
status.drop(['Churn Category'],axis=1,inplace=True)
status.rename(columns={'ans':'Churn Category'}, inplace=True)
#final train data
train=pd.merge(status,users,on='Customer ID',how='left')

# Train Model

In [9]:
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC 

In [10]:
features=list(train)
print(features)
features.remove('Customer ID')
features.remove('Churn Category')
features.remove('Referred_a_Friend')
features.remove('Offer')
'''
features.remove('Under 30')
features.remove('Married')
features.remove('Senior Citizen')
features.remove('Dependents')
features.remove('City')
features.remove('Referred a Friend')
features.remove('Offer')
features.remove('Phone Service')
features.remove('Multiple Lines')
features.remove('Internet Service')
features.remove('Internet Type')
features.remove('Online Security')
features.remove('Online Backup')
features.remove('Device Protection Plan')
features.remove('Premium Tech Support')
features.remove('Streaming TV')
features.remove('Streaming Movies')
features.remove('Streaming Music')
features.remove('Unlimited Data')
features.remove('Contract')
features.remove('Paperless Billing')
features.remove('Payment Method')
'''
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
imputer = imputer.fit(train.loc[:, features])
train_imputed = imputer.fit_transform(train.loc[:, features])
df=pd.DataFrame(train_imputed)
poly = PolynomialFeatures(degree=2, interaction_only=True)
train_imputed = pd.DataFrame(poly.fit_transform(df))
print(train_imputed.shape)


['Customer ID', 'Churn Category', 'Age', 'Number_of_Dependents', 'Zip Code', 'Population', 'score', 'Referred_a_Friend', 'Number of Referrals', 'Tenure in Months', 'Offer', 'Avg Monthly Long Distance Charges', 'Avg Monthly GB Download', 'Monthly Charge', 'Total Charges', 'Total Refunds', 'Total Extra Data Charges', 'Total Long Distance Charges', 'Total Revenue', 'Married_No', 'Married_Yes', 'Gender_Female', 'Gender_Male', 'Under_30_No', 'Under_30_Yes', 'Senior_Citizen_No', 'Senior_Citizen_Yes', 'Dependents_No', 'Dependents_Yes', 'City_Acampo', 'City_Acton', 'City_Adelanto', 'City_Adin', 'City_Agoura Hills', 'City_Aguanga', 'City_Ahwahnee', 'City_Alameda', 'City_Alamo', 'City_Albany', 'City_Albion', 'City_Alderpoint', 'City_Alhambra', 'City_Aliso Viejo', 'City_Alleghany', 'City_Alpaugh', 'City_Alpine', 'City_Alta', 'City_Altadena', 'City_Alturas', 'City_Alviso', 'City_Amador City', 'City_Amboy', 'City_Anaheim', 'City_Anderson', 'City_Angels Camp', 'City_Angelus Oaks', 'City_Angwin', 'Ci

In [11]:
# 10% vaildation 
x_train,x_test,y_train,y_test = train_test_split(train_imputed,train.loc[:, 'Churn Category'],test_size=0.90, random_state=0)

### SVM_linear 

In [12]:
best_acc=0
best_c=0
best_g=0
best_d=0
tc=1
for c in range (1):
    tg=1
    for g in range(1):
        for d in range(3,4):
            print(c,g,d)
            svm_model = SVC(kernel = 'rbf',degree=d,gamma=tg, C = tc).fit(x_train, y_train)
            svm_predictions = svm_model.predict(x_test)
            accuracy = svm_model.score(x_test, y_test)
            if(accuracy>best_acc):
                best_acc=accuracy
                best_c=tc
                best_g=tg
                best_d=d
        tg*=10
    tc*=10
print(best_acc)
svm_model = SVC(kernel = 'rbf',degree=best_d,gamma=best_g, C = best_c).fit(train_imputed, train.loc[:,'Churn Category'])

0 0 3


### decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier(max_depth = 1).fit(x_train, y_train)
dtree_predictions = dtree_model.predict(x_test)
accuracy = dtree_model.score(x_test, y_test)
print(accuracy)

0.8091482649842271


### knn 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 38).fit(x_train, y_train)
accuracy = knn.score(x_test, y_test)
print(accuracy)


0.7389589905362776


### RandomForest + ada boost

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

rf = RandomForestClassifier(n_estimators = 1000, random_state = 0,n_jobs=-1)
ada = AdaBoostClassifier(n_estimators = 1000, random_state = 0, )



rf.fit(x_train, y_train)
rf.score(x_train, y_train)

#ada.fit(x_train, y_train)
#ada.score(x_train, y_train)

1.0

In [ ]:
rf.score(x_test, y_test)

0.7628811777076762

### Prediction

In [ ]:
testID=pd.read_csv('Test_IDs.csv')
testID.columns=['Customer ID']
dftest=pd.DataFrame(testID)
test=pd.merge(dftest,users,on='Customer ID',how='left')
#print(test)
test_imputed = imputer.fit_transform(test.loc[:, features])
df=pd.DataFrame(test_imputed)
poly = PolynomialFeatures(degree=2, interaction_only=True)
test_imputed = pd.DataFrame(poly.fit_transform(df))
#print(train_imputed.shape)
print(test_imputed)
dftest['Churn Category']=svm_model.predict(test_imputed)

#dftest['Churn Category']=dtree_model.predict(test_imputed)

#testID['Churn Category']=knn.predict(test_imputed)

#dftest['Churn Category']=rfr.predict(test_imputed)
#dftest['Churn Category']=rf.predict(test_imputed)
#dftest['Churn Category']=ada.predict(test_imputed)

[[7.3000e+01 0.0000e+00 9.2027e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [6.4000e+01 0.0000e+00 9.3532e+04 ... 1.0000e+00 0.0000e+00 0.0000e+00]
 [4.7000e+01 0.0000e+00 9.2119e+04 ... 0.0000e+00 1.0000e+00 0.0000e+00]
 ...
 [2.7000e+01 0.0000e+00 9.3532e+04 ... 0.0000e+00 0.0000e+00 0.0000e+00]
 [3.8000e+01 3.0000e+00 9.2604e+04 ... 0.0000e+00 1.0000e+00 0.0000e+00]
 [4.7000e+01 0.0000e+00 9.0301e+04 ... 0.0000e+00 1.0000e+00 0.0000e+00]]


### Output result

In [ ]:
testID.columns=['Customer ID','Churn Category']
submiss=pd.DataFrame(testID)
submiss.to_csv('submission.csv',index=False)